In [1]:
import pandas as pd
import altair as alt

In [2]:
dados_federais_campina = pd.read_csv("data\\dados_casos_obitos_cg.csv", encoding='UTF-8', error_bad_lines=False)
dados_municipais_campina = pd.read_csv("data\\dados_municipais_cg.csv", encoding='utf-8', index_col='data')

C:\Users\perei\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


# Gerando dados sobre o Numero de Obitos por Dia (Dados Federais)

In [3]:
# Criando um data com a coluna 'data' e a coluna 'obitosNovos'
df_obitosPorDia = dados_federais_campina[['obitosNovos', 'data']]

In [4]:
# Criando um DataFrame com os dados para o altair
df_obitosPorDia = pd.DataFrame({
    'data': df_obitosPorDia['data'].tolist(),
    'obitos_confirmados': df_obitosPorDia['obitosNovos'].tolist()
})
# Criando uma nova coluna no DataFrame referente a média movel.
df_obitosPorDia['media_movel'] = df_obitosPorDia.obitos_confirmados.rolling(7).mean()

In [5]:
# Gerando o Gráfico de Obitos por Dia com os dados Federais
graph_df_obitosPorDia = alt.Chart(df_obitosPorDia).mark_line().encode(
    alt.X('data:T', title='Data'),
    alt.Y('obitos_confirmados:Q', title='Obitos Confirmados'),
    color=alt.value('dodgerblue')
).properties(
    width=800,
    title='Obitos Confirmados por Dia (Base Federal)'
)

graph_df_obitosPorDia

alt.Chart(...)

In [6]:
# Gerando o grafico de média movel
graph_df_obitosMedMovel = alt.Chart(df_obitosPorDia).mark_line(
    point=True
).encode(
    alt.X('data:T', title='Data'),
    alt.Y('media_movel:Q', title='Obitos Confirmados'),
    color=alt.value('orange')
)

In [7]:
graph_df_omm = graph_df_obitosPorDia + graph_df_obitosMedMovel
graph_df_omm

alt.LayerChart(...)

# Gerando dados sobre o Numero de Obitos por Dia (Dados Municipais)

In [8]:
# Criando um data com a coluna 'data' e a coluna 'obitosNovos'
dm_obitosPorDia = dados_municipais_campina['obitosPorDia']

In [9]:
# Criando um DataFrame com os dados para o altair
dm_obitosPorDia = pd.DataFrame({
    'data': dm_obitosPorDia.index.tolist(),
    'obitos_confirmados': dm_obitosPorDia.values.tolist()
})
# Criando uma nova coluna no DataFrame referente a média movel.
dm_obitosPorDia['media_movel'] = dm_obitosPorDia.obitos_confirmados.rolling(7).mean()

In [10]:
graph_dm_obitosPorDia = alt.Chart(dm_obitosPorDia).mark_line().encode(
    alt.X('data:T', title='Data'),
    alt.Y('obitos_confirmados:Q', title='Obitos Confirmados'),
    color=alt.value('darkorange')
).properties(
    width=800,
    title='Obitos Confirmados por Dia (Base Municipal)'
)

graph_dm_obitosPorDia

alt.Chart(...)

In [11]:
# Gerando o grafico de média movel
graph_dm_obitosMedMovel = alt.Chart(dm_obitosPorDia).mark_line(
    point=True
).encode(
    alt.X('data:T', title='Data'),
    alt.Y('media_movel:Q', title='Obitos Confirmados'),
    color=alt.value('black')
)

In [12]:
graph_dm_omm = graph_dm_obitosPorDia + graph_dm_obitosMedMovel
graph_dm_omm

alt.LayerChart(...)

# Comparando os dados Federais e Municipais

In [13]:
graph_comp_obitosPorDia = graph_df_obitosPorDia + graph_dm_obitosPorDia
graph_comp_obitosPorDia.properties(
    title='Obitos Confirmados Por Dia (Comparação)'
)

alt.LayerChart(...)

In [14]:
graph_comp_mediamovel = graph_df_obitosMedMovel + graph_dm_obitosMedMovel
graph_comp_mediamovel.properties(
    width=800,
    title='Média Movel de Obitos Confirmados por Dia (Comparação)'
)

alt.LayerChart(...)

In [15]:
df_obitosTotais = dados_federais_campina['obitosAcumulado'].iloc[-1]
dm_obitosTotais = dados_municipais_campina['obitosAcumulados'].iloc[-1]

In [16]:
totalObitos = pd.DataFrame({
    'base': ['Federal', 'Municipal'],
    'total': [df_obitosTotais, dm_obitosTotais]
})

In [17]:
alt.Chart(totalObitos).mark_bar(size=30).encode(
    alt.X('base:N', title='Base dos Dados'),
    alt.Y('total:Q', title='Nº Total de Obitos Confirmados'),
    alt.Color('base:N', title='Instância')
).properties(
    width=150,
    title='Total de Obitos Confirmados (Comparação)'
)

alt.Chart(...)